In [1]:
!pip install --upgrade transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.5/511.5 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 21.1 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
pylibcudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.


In [ ]:
import pandas as pd
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import numpy as np
import torch

In [ ]:
# Controlla se è disponibile una GPU (MOLTO raccomandato)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Utilizzo del dispositivo: {device}")

In [ ]:
# --- 1. CARICARE I SET DI DATI ---
try:
    train_df = pd.read_csv("train_set.csv")
    val_df = pd.read_csv("validation_set.csv")
    test_df = pd.read_csv("test_set.csv")
except FileNotFoundError:
    print("ERRORE: File di set non trovati.")
    print("Esegui prima 'split_data.py'.")
    exit()

# Converti i DataFrame di Pandas in 'Dataset' di Hugging Face
ds = DatasetDict({
    'train': Dataset.from_pandas(train_df),
    'validation': Dataset.from_pandas(val_df),
    'test': Dataset.from_pandas(test_df)
})

print(f"Dataset caricati:\n{ds}")

In [ ]:
# --- 2. CARICARE TOKENIZER E MODELLO ---
MODEL_NAME = "microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract"

# Carica il Tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Carica il Modello
# num_labels=2 (classe 0 e classe 1)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)
model.to(device)  # Sposta il modello sulla GPU se disponibile

In [ ]:
# --- 3. TOKENIZZARE I DATI ---
def tokenize_function(examples):
    # 'truncation=True' taglia i testi più lunghi di 512 token
    # 'padding="max_length"' aggiunge token finti fino a 512
    return tokenizer(examples['text'], padding="max_length", truncation=True, max_length=512)


# Applica la tokenizzazione a tutti i set in parallelo
tokenized_datasets = ds.map(tokenize_function, batched=True)

# Rimuovi la colonna 'text' (non più necessaria) e formatta per il training
tokenized_datasets = tokenized_datasets.remove_columns(["text"])
tokenized_datasets.set_format("torch")



In [ ]:
import transformers
import datasets

print(f"Transformers version: {transformers.__version__}")
print(f"Datasets version: {datasets.__version__}")

In [ ]:

# --- 4. DEFINIRE LE METRICHE DI VALUTAZIONE ---
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    acc = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='binary')
    precision = precision_score(labels, predictions, average='binary')
    recall = recall_score(labels, predictions, average='binary')

    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
# --- 5. DEFINIRE GLI ARGOMENTI DI TRAINING ---
training_args = TrainingArguments(
    output_dir="./polyphenol_classifier",  # Cartella dove salvare il modello
    eval_strategy="epoch",  # Valuta alla fine di ogni epoca
    save_strategy="epoch",  # Salva il modello alla fine di ogni epoca
    num_train_epochs=3,  # 3 epoche sono un buon punto di partenza
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=2e-5,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    load_best_model_at_end=True,  # Carica il modello migliore alla fine
    metric_for_best_model="f1",  # Scegli il modello migliore in base all'F1-score
    report_to="none"  # Disabilita il logging online (wandb)
)

In [ ]:
# --- 6. CREARE IL TRAINER ---
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

In [ ]:
# --- 7. AVVIARE IL TRAINING (FINE-TUNING) ---
print("\n--- INIZIO FINE-TUNING MODELLO ---")
trainer.train()
print("--- FINE-TUNING COMPLETATO ---")

In [ ]:
# --- 8. VALUTAZIONE FINALE SUL TEST SET ---
print("\n--- VALUTAZIONE SUL TEST SET (DATI MAI VISTI) ---")
test_results = trainer.evaluate(eval_dataset=tokenized_datasets["test"])

print(f"Accuracy sul Test Set: {test_results['eval_accuracy']:.4f}")
print(f"F1-score sul Test Set: {test_results['eval_f1']:.4f}")
print(f"Precision sul Test Set: {test_results['eval_precision']:.4f}")
print(f"Recall sul Test Set: {test_results['eval_recall']:.4f}")

In [ ]:
print("\n--- OTTENIMENTO DEI LOGITS E DELLE PROBABILITÀ ---")
predictions = trainer.predict(tokenized_datasets["test"])

logits = predictions.predictions   # array di dimensione [num_samples, 2]
labels = predictions.label_ids     # etichette vere (0 o 1)

# Converti i logits in probabilità con softmax
probs = torch.softmax(torch.tensor(logits), dim=1)

# Estrai la probabilità della classe "rilevante" (indice 1)
relevance_scores = probs[:, 1].numpy()

# Mostra qualche esempio
for i in range(5):
    print(f"Esempio {i+1}:")
    print(f"  Logits grezzi: {logits[i]}")
    print(f"  Probabilità rilevante: {relevance_scores[i]:.4f}")
    print(f"  Etichetta vera: {labels[i]}")

In [ ]:

# Salva i risultati finali su un file
with open("test_results.txt", "w") as f:
    f.write(str(test_results))

# Salva il modello finale
trainer.save_model("./polyphenol_classifier_final")
print("Modello finale salvato in './polyphenol_classifier_final'")

In [ ]:
!zip -r modello_finito.zip ./polyphenol_classifier_final

In [ ]:
from sklearn.metrics import average_precision_score, ndcg_score
import numpy as np

# --- METRICHE RANKING-BASED ---
map_score = average_precision_score(labels, relevance_scores)

def mean_reciprocal_rank(y_true, y_score):
    sorted_indices = np.argsort(-y_score)
    for rank, idx in enumerate(sorted_indices, start=1):
        if y_true[idx] == 1:
            return 1.0 / rank
    return 0.0

mrr_score = mean_reciprocal_rank(labels, relevance_scores)
ndcg = ndcg_score([labels], [relevance_scores])

print("\n--- Ranking-based Metrics ---")
print(f"MAP: {map_score:.4f}")
print(f"MRR: {mrr_score:.4f}")
print(f"nDCG: {ndcg:.4f}")